## libraries

In [1]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2 as cv
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Input, Activation, Add, Conv2D, Multiply, MaxPooling2D, UpSampling2D, Dropout, concatenate, Lambda
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.layers as L

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import keras.backend as K

from imgaug import augmenters as iaa
import scipy.ndimage as ndimage


## Metrics and functions

In [2]:
def get_f1(y_true, y_pred):
    # Round predictions to binary values (0 or 1)
    y_pred = tf.round(y_pred)

    # Calculate True Positives, False Positives, and False Negatives
    true_positives = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
    possible_positives = tf.reduce_sum(tf.cast(y_true, tf.float32))
    predicted_positives = tf.reduce_sum(tf.cast(y_pred, tf.float32))

    # Calculate Precision and Recall
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())

    # Calculate F1 Score
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    return f1

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="tp", initializer="zeros")
        self.false_positives = self.add_weight(name="fp", initializer="zeros")
        self.false_negatives = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)
        self.true_positives.assign_add(tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32)))
        self.false_positives.assign_add(tf.reduce_sum(tf.cast((1 - y_true) * y_pred, tf.float32)))
        self.false_negatives.assign_add(tf.reduce_sum(tf.cast(y_true * (1 - y_pred), tf.float32)))

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_state(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)


def sobel_edge_detection(x):
    sobel_x = tf.image.sobel_edges(x)
    edges = tf.sqrt(sobel_x[..., 0] ** 2 + sobel_x[..., 1] ** 2)  # Combine the gradients in X and Y directions
    return edges

one_weight = 0.55
zero_weight = 0.45
def create_weighted_binary_cross_entropy(zero_weight, one_weight):
    def weighted_binary_crossentropy(y_true, y_pred):
        # Calculate the binary crossentropy
        b_ce = K.binary_crossentropy(y_true, y_pred)

        # Apply the weights
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce

        # Return the mean error
        return K.mean(weighted_b_ce)

    return weighted_binary_crossentropy

def attention_gate(g, s, num_filters):
    Wg = L.Conv2D(num_filters, 1, padding="same")(g)
    Wg = L.BatchNormalization()(Wg)

    Ws = L.Conv2D(num_filters, 1, padding="same")(s)
    Ws = L.BatchNormalization()(Ws)

    out = L.Activation("relu")(Wg + Ws)
    out = L.Conv2D(num_filters, 1, padding="same")(out)
    out = L.Activation("sigmoid")(out)

    return out * s


## eagle loss function

## model

In [3]:
def fat_unet(input_size, verbose = False):
    #Taken from https://github.com/zhixuhao/unet/blob/master/model.py
    inputs = Input(input_size)
    s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(s)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    drop3 = Dropout(0.3)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    attention6 = attention_gate(drop4, up6, 512)
    merge6 = concatenate([attention6,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    attention7 = attention_gate(drop3, up7, 256)
    merge7 = concatenate([attention7, up7], axis=3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    attention8 = attention_gate(conv2, up8, 128)
    merge8 = concatenate([attention8, up8], axis=3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    attention9 = attention_gate(conv1, up9, 64)
    merge9 = concatenate([attention9, up9], axis=3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    # Edge detection to enhance segmentation
    edges = sobel_edge_detection(conv9)  # Apply edge detection to the last conv feature map

    # Combine edge information with segmentation output
    seg_output = Conv2D(1, 1, activation='sigmoid', name="segmentation_output")(conv9)
    
    enhanced_seg_output = tf.keras.layers.Multiply()([seg_output, edges])  # Use edge features to refine segmentation

    # Final model output
    model = Model(inputs=inputs, outputs=enhanced_seg_output, name='fat_unet')
    
    loss = create_weighted_binary_cross_entropy(zero_weight, one_weight)
    
    model.load_weights(r'C:/Users/LENOVO/Personal Projects/deep project/checkpoints/experiment82/experiment82.weights.h5')  # Load weights into the model
    
    model.compile(optimizer = Adam(learning_rate = 1e-4), loss = loss, metrics = [F1Score()])

    if verbose:
        model.summary()

    return model

## Data loading

In [4]:
image_dir = r"C:\Users\LENOVO\Personal Projects\deep project\dataset\training\training orig\images"
mask_dir = r"C:\Users\LENOVO\Personal Projects\deep project\dataset\training\training orig\groundtruth"

input_size = (256, 256, 3)

# data loading function
def load_data(image_dir, mask_dir, input_size):
    images, masks = [], []

    for image_name in tqdm(os.listdir(image_dir), desc="Processing Images"):
        # Load and preprocess image
        img_path = os.path.join(image_dir, image_name)
        img = load_img(img_path, target_size=input_size[:2])
        img = img_to_array(img) / 255.0  # Normalize

        # Load and preprocess mask
        mask_path = os.path.join(mask_dir, image_name)  # Assuming masks have the same name
        mask = load_img(mask_path, target_size=input_size[:2], color_mode='grayscale')
        mask = img_to_array(mask) / 255.0  # Normalize

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

images, masks = load_data(image_dir, mask_dir, input_size)

print(images.shape)

# test split
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.35, random_state=42)

print(X_train.shape, y_train.shape)

Processing Images: 100%|████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 180.44it/s]

(100, 256, 256, 3)
(65, 256, 256, 3) (65, 256, 256, 1)


In [5]:
def get_rotations_90_180_270(images, masks):
    rotated_images, rotated_masks = [], []

    for img, mask in zip(images, masks):

        # Generate and append the rotations
        for k in [1, 2, 3]:  # 90°, 180°, and 270° rotations
            rotated_images.append(np.rot90(img, k=k, axes=(0, 1)))
            rotated_masks.append(np.rot90(mask, k=k, axes=(0, 1)))

    return np.array(rotated_images), np.array(rotated_masks)

# Function to apply augmentations (rotations in this case)
def apply_augments_15(images, masks):
    """Applies affine rotation augmentations (same for images and masks)."""
    augmenter = iaa.Sequential(iaa.Affine(rotate=(-15, 15)))  # Rotate within [-45°, 45°]

    # Ensure deterministic augmentations
    deterministic_aug = augmenter.to_deterministic()

    # Apply to both images and masks
    augmented_images = deterministic_aug(images=images)
    augmented_masks = deterministic_aug(images=masks)  # Same augmentations as images

    return np.array(augmented_images), np.array(augmented_masks)

def apply_augments_30(images, masks):
    """Applies affine rotation augmentations (same for images and masks)."""
    augmenter = iaa.Sequential(iaa.Affine(rotate=(-30, 30)))  # Rotate within [-45°, 45°]

    # Ensure deterministic augmentations
    deterministic_aug = augmenter.to_deterministic()

    # Apply to both images and masks
    augmented_images = deterministic_aug(images=images)
    augmented_masks = deterministic_aug(images=masks)  # Same augmentations as images

    return np.array(augmented_images), np.array(augmented_masks)

def apply_augments_45(images, masks):
    """Applies affine rotation augmentations (same for images and masks)."""
    augmenter = iaa.Sequential(iaa.Affine(rotate=(-45, 45)))  # Rotate within [-45°, 45°]

    # Ensure deterministic augmentations
    deterministic_aug = augmenter.to_deterministic()

    # Apply to both images and masks
    augmented_images = deterministic_aug(images=images)
    augmented_masks = deterministic_aug(images=masks)  # Same augmentations as images

    return np.array(augmented_images), np.array(augmented_masks)

def apply_augments_60(images, masks):
    """Applies affine rotation augmentations (same for images and masks)."""
    augmenter = iaa.Sequential(iaa.Affine(rotate=(-60, 60)))  # Rotate within [-45°, 45°]

    # Ensure deterministic augmentations
    deterministic_aug = augmenter.to_deterministic()

    # Apply to both images and masks
    augmented_images = deterministic_aug(images=images)
    augmented_masks = deterministic_aug(images=masks)  # Same augmentations as images

    return np.array(augmented_images), np.array(augmented_masks)

# Apply augmentation to training data only
augmented_x1, augmented_y1 = apply_augments_15(X_train, y_train)

augmented_x2, augmented_y2 = apply_augments_30(X_train, y_train)

augmented_x3, augmented_y3 = apply_augments_45(X_train, y_train)

augmented_x4, augmented_y4 = apply_augments_60(X_train, y_train)

rotated_images, rotated_masks = get_rotations_90_180_270(X_train, y_train)

X_train = np.concatenate((X_train, rotated_images), axis=0)
y_train = np.concatenate((y_train, rotated_masks), axis=0)
X_train = np.concatenate([X_train, augmented_x1], axis=0)
y_train = np.concatenate([y_train, augmented_y1], axis=0)
X_train = np.concatenate([X_train, augmented_x2], axis=0)
y_train = np.concatenate([y_train, augmented_y2], axis=0)
X_train = np.concatenate([X_train, augmented_x3], axis=0)
y_train = np.concatenate([y_train, augmented_y3], axis=0)
X_train = np.concatenate([X_train, augmented_x4], axis=0)
y_train = np.concatenate([y_train, augmented_y4], axis=0)

print(X_train.shape, y_train.shape)

(520, 256, 256, 3) (520, 256, 256, 1)


In [6]:
# train split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(X_train.shape)

(416, 256, 256, 3)


## Model training

In [7]:
CHECKPOINT_PATH = r"C:/Users/LENOVO/Personal Projects/deep project/checkpoints/"

def create_new_experiment_folder(base_path):

    # Get the list of existing folders
    existing_folders = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

    # Filter folders that match the experiment pattern
    experiment_numbers = []
    for folder in existing_folders:
        if folder.startswith("experiment"):
            try:
                num = int(folder[len("experiment"):])
                experiment_numbers.append(num)
            except ValueError:
                continue

    # Determine the next experiment number
    next_experiment_number = max(experiment_numbers, default=0) + 1

    # Create the new experiment folder
    directory_name = f"experiment{next_experiment_number}"
    new_folder_name = f"{directory_name}/experiment{next_experiment_number}.weights.h5"
    new_folder_path = os.path.join(base_path, new_folder_name)
    try:
        os.mkdir(os.path.join(base_path,directory_name))
    except Exception as e:
        print(f"An error occurred: {e}")
    return new_folder_path

CHECKPOINT_PATH = create_new_experiment_folder(CHECKPOINT_PATH)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.85, patience=3, verbose=1, min_lr=1e-6)


callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=7, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH, save_weights_only=True, verbose=1, save_best_only=True),
        reduce_lr
]

In [8]:
input_size = X_train.shape[1:]

# Create the model
model = fat_unet(input_size=input_size, verbose=True)

# Train the model
history = model.fit(
    X_train,y_train,
    batch_size= 4,
    epochs=60,
    validation_data=(X_val, y_val), 
    callbacks=callbacks
)

Model: "fat_unet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 256, 256, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['lambda[0][0]']                 
                                )                                                                 
                                                                                           

## Model test

In [ ]:
with tf.device('/CPU:0'):

    test_loss, test_f1 = model.evaluate(X_test, y_test)

    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test F1 Score: {test_f1:.4f}")

    # Predict on the test set
    y_pred = model.predict(X_test, verbose=1)

    # Post-process predictions
    # Threshold predictions for binary masks
    y_pred_thresholded = (y_pred > 0.5).astype(np.uint8)

    # Example: Visualize predictions alongside ground truth
    import matplotlib.pyplot as plt

    def display_predictions(X_test, y_test, y_pred, num_samples=5):
        """Displays test images, ground truth, and predictions side by side."""
        for i in range(num_samples):
            plt.figure(figsize=(12, 4))

            # Display the test image
            plt.subplot(1, 3, 1)
            plt.imshow(X_test[i])
            plt.title("Test Image")
            plt.axis("off")

            # Display the ground truth mask
            plt.subplot(1, 3, 2)
            plt.imshow(y_test[i].squeeze(), cmap="gray")
            plt.title("Ground Truth")
            plt.axis("off")

            # Display the predicted mask
            plt.subplot(1, 3, 3)
            plt.imshow(y_pred[i].squeeze(), cmap="gray")
            plt.title("Predicted Mask")
            plt.axis("off")

            plt.show()

In [ ]:
def display_predictions(X_test, y_test, y_pred, num_samples=5):
    """
    Display test samples, their ground truth, and model predictions.
    """
    for i in range(num_samples):
        plt.figure(figsize=(15, 5))

        # Display the input image
        plt.subplot(1, 3, 1)
        plt.imshow(X_test[i].squeeze(), cmap="gray")
        plt.title("Input Image")
        plt.axis("off")

        # Display the ground truth mask
        plt.subplot(1, 3, 2)
        plt.imshow(y_test[i].squeeze(), cmap="gray")
        plt.title("Ground Truth")
        plt.axis("off")

        # If the prediction has multiple channels, display each channel
        if y_pred[i].shape[-1] > 1:
            num_channels = y_pred[i].shape[-1]
            plt.figure(figsize=(15, 5))  # Adjust the figure size as needed
            for channel in range(num_channels):
                plt.subplot(1, num_channels, channel + 1)
                plt.imshow(y_pred[i][..., channel], cmap="gray")
                plt.title(f"Channel {channel}")
                plt.axis("off")
        else:
            # Display the single channel prediction mask
            pred_mask = y_pred[i].squeeze()
            plt.imshow(pred_mask, cmap="gray")
            plt.title("Predicted Mask")
            plt.axis("off")

        plt.show()

with tf.device('/CPU:0'):
    display_predictions(X_test, y_test, y_pred_thresholded, num_samples=20)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
with tf.device('/CPU:0'): 
    # Load test images using the modified function
    def load_test_data(image_dir, input_size):
        images = []
        for folder_name in os.listdir(image_dir):
            folder_path = os.path.join(image_dir, folder_name)

            # Check if it's a directory
            if os.path.isdir(folder_path):
                for image_name in os.listdir(folder_path):
                    img_path = os.path.join(folder_path, image_name)

                    # Only process if it's a .png file (or you can include other formats)
                    if os.path.isfile(img_path) and image_name.endswith('.png'):
                        # Load and preprocess image
                        img = load_img(img_path, target_size=input_size[:2])  # Resize to match input size
                        img = img_to_array(img)  # Convert to NumPy array
                        img = img / 255.0  # Normalize to [0, 1]
                        images.append(img)

        return np.array(images)

    # Path to test images directory
    image_dir = r"C:\Users\LENOVO\Personal Projects\deep project\dataset\test_set_images\test_set_images"

    # Define the input size (same as training)
    input_size = (256, 256, 3)

    # Load test images
    test_images = load_test_data(image_dir, input_size)
    print(f"Test images shape: {test_images.shape}")  # Check the shape

    # Run predictions
    predictions = model.predict(test_images)

    # Post-process the predictions (assuming binary segmentation)
    # Convert the predicted probabilities to binary mask
    predicted_masks = (predictions > 0.5).astype(np.uint8)  # Apply a 0.5 threshold
"""
    # Visualize the input images and their predicted masks
    for i in range(len(test_images)):
        original_image = test_images[i]
        predicted_mask = predicted_masks[i]

        plt.figure(figsize=(10, 5))

        # Original image
        plt.subplot(1, 2, 1)
        plt.title(f"Original Image {i+1}")
        plt.imshow(original_image.squeeze())
        plt.axis('off')

        # Use predictions for the predicted mask
        predicted_mask = np.argmax(predictions[i], axis=-1)  # Apply to 'predictions', not 'y_pred'

        # Handle multi-channel case (if applicable)
        if predicted_mask.ndim == 3:  # Multiple channels
            num_channels = predicted_mask.shape[-1]

            # Create a grid to display all channels
            plt.figure(figsize=(15, 5))
            for j in range(num_channels):
                plt.subplot(1, num_channels, j + 1)
                plt.imshow(predicted_mask[:, :, j].squeeze(), cmap="gray")
                plt.title(f"Channel {j + 1}")
                plt.axis("off")
        else:  # Single-channel case
            plt.figure(figsize=(5, 5))
            plt.imshow(predicted_mask.squeeze(), cmap="gray")
            plt.title("Predicted Mask")
            plt.axis("off")

        plt.show()
        """

In [ ]:
# Visualize the input images and their predicted masks
for i in range(len(test_images)):
    original_image = test_images[i]
    predicted_mask = predicted_masks[i]

    plt.figure(figsize=(10, 5))

    # Original image
    plt.subplot(1, 2, 1)
    plt.title(f"Original Image {i+1}")
    plt.imshow(original_image)
    plt.axis('off')

    if predicted_mask.ndim == 3:  # Multiple channels
        num_channels = predicted_mask.shape[-1]

        # Create a grid to display all channels
        plt.figure(figsize=(15, 5))
        for i in range(num_channels):
            plt.subplot(1, num_channels, i + 1)
            plt.imshow(predicted_mask[:, :, i], cmap="gray")
            plt.title(f"Channel {i + 1}")
            plt.axis("off")
    else:  # Single-channel case
        plt.figure(figsize=(5, 5))
        plt.imshow(predicted_mask, cmap="gray")
        plt.title("Predicted Mask")
        plt.axis("off")

    plt.show()

In [ ]:
model.summary()


In [ ]:
with tf.device('/CPU:0'): 

    # Visualize the input images and their predicted masks
    for i in range(len(test_images)):
        original_image = test_images[i]
        predicted_mask = predicted_masks[i]

        plt.figure(figsize=(10, 5))

        # Original image
        plt.subplot(1, 2, 1)
        plt.title(f"Original Image {i+1}")
        plt.imshow(original_image.squeeze())
        plt.axis('off')

        # Use predictions for the predicted mask
        #predicted_mask = np.argmax(predictions[i], axis=-1)  # Apply to 'predictions', not 'y_pred'

        # Handle multi-channel case (if applicable)
        if predicted_mask.ndim == 3:  # Multiple channels
            num_channels = predicted_mask.shape[-1]

            # Create a grid to display all channels
            plt.figure(figsize=(10, 5))
            for j in range(num_channels):
                plt.subplot(1, num_channels, j + 1)
                plt.imshow(predicted_mask[:, :, j].squeeze(), cmap="gray")
                plt.title(f"Channel {j + 1}")
                plt.axis("off")
        else:  # Single-channel case
            plt.figure(figsize=(10, 5))
            plt.imshow(predicted_mask.squeeze(), cmap="gray")
            plt.title("Predicted Mask")
            plt.axis("off")

        plt.show()